#### First get the candidate model parts (Head and Feature Extractor)

In [1]:
import mlflow
mlflow.set_tracking_uri("http://127.0.0.1:5000")
model_uri = "models:/{}/{}".format('Laion Balanced', '1')
head_model = mlflow.pytorch.load_model(model_uri)
head_model

2023/04/16 19:19:28 WARNING mlflow.pytorch: Stored model version '1.13.1' does not match installed PyTorch version '2.0.0'


DiffFeatureDetectorParamBiDirectional(
  (cls_layer): Sequential(
    (features_dropout): Dropout(p=0.31296189949335906, inplace=False)
    (linear_0): Linear(in_features=1280, out_features=184, bias=True)
    (relu_0): ReLU()
    (hidden_dropout_0): Dropout(p=0.5838650129135917, inplace=False)
    (linear_1): Linear(in_features=184, out_features=184, bias=True)
    (relu_1): ReLU()
    (hidden_dropout_1): Dropout(p=0.5838650129135917, inplace=False)
    (linear_2): Linear(in_features=184, out_features=184, bias=True)
    (relu_2): ReLU()
    (hidden_dropout_2): Dropout(p=0.5838650129135917, inplace=False)
    (linear_out): Linear(in_features=184, out_features=1, bias=True)
  )
)

In [10]:
from drecg.training.ignite_finetune import define_model_for_tune
import mlflow
mlflow.set_tracking_uri("http://127.0.0.1:5000")
model_1 = define_model_for_tune('Laion Balanced', 'ViT_LAION')

2023/04/16 22:29:37 WARNING mlflow.pytorch: Stored model version '1.13.1' does not match installed PyTorch version '2.0.0'
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
2023-04-16 22:30:01.330992: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-16 22:30:01.493154: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-16 22:30:02.036493: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
model_1

In [8]:
type(model.feature_extractor.vit_model.text_model), type(model.feature_extractor.vit_model.vision_model)

(transformers.models.clip.modeling_clip.CLIPTextTransformer,
 transformers.models.clip.modeling_clip.CLIPVisionTransformer)

In [1]:
from transformers.models.clip.modeling_clip import CLIPModel
CLIPModel().push_to_hub

transformers.models.clip.modeling_clip.CLIPModel

In [10]:
model.feature_extractor.vit_model.text_model.parameters()

<generator object Module.parameters at 0x7fced6f825e0>

In [14]:
def count_parameters(params):
    return sum(p.numel() for p in params)

text_params = count_parameters(model.feature_extractor.vit_model.text_model.parameters())

In [13]:
vision_params = count_parameters(model.feature_extractor.vit_model.vision_model.parameters())

In [15]:
text_params

693021440

In [19]:
vision_params

1842777344

In [20]:
bytes_per_param = 4
vision_bytes = vision_params * bytes_per_param

In [21]:
vision_gigabytes = vision_bytes / 1024 / 1024 / 1024

In [22]:
vision_gigabytes

6.864880561828613

In [23]:
text_gigabytes = text_params * bytes_per_param / 1024 / 1024 / 1024

In [24]:
text_gigabytes

2.5817060470581055

In [1]:
from huggingface_hub import PyTorchModelHubMixin

In [2]:
from transformers import AutoModel

vit_model = AutoModel.from_pretrained("laion/CLIP-ViT-bigG-14-laion2B-39B-b160k")

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
type(vit_model)

transformers.models.clip.modeling_clip.CLIPModel

In [6]:
from transformers.models.clip.modeling_clip import CLIPModel

In [4]:
model.feature_extractor

VitLaionFeatureExtractor(
  (transforms): VitLaionPreProcess()
  (vision_model): CLIPVisionTransformer(
    (embeddings): CLIPVisionEmbeddings(
      (patch_embedding): Conv2d(3, 1664, kernel_size=(14, 14), stride=(14, 14), bias=False)
      (position_embedding): Embedding(257, 1664)
    )
    (pre_layrnorm): LayerNorm((1664,), eps=1e-05, elementwise_affine=True)
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0-47): 48 x CLIPEncoderLayer(
          (self_attn): CLIPAttention(
            (k_proj): Linear(in_features=1664, out_features=1664, bias=True)
            (v_proj): Linear(in_features=1664, out_features=1664, bias=True)
            (q_proj): Linear(in_features=1664, out_features=1664, bias=True)
            (out_proj): Linear(in_features=1664, out_features=1664, bias=True)
          )
          (layer_norm1): LayerNorm((1664,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): GELUActivation()
            (fc1): Linear(i

In [1]:
from typing import Optional
from transformers.models.clip.modeling_clip import CLIPVisionTransformer
from torch import nn
import torch
from transformers.models.clip.configuration_clip import CLIPVisionConfig
from huggingface_hub import PyTorchModelHubMixin

class VitImageFeatureExtractor(nn.Module, PyTorchModelHubMixin):
    def __init__(self):
        super().__init__()

        config = {
          "attention_dropout": 0.0,
          "dropout": 0.0,
          "hidden_act": "gelu",
          "hidden_size": 1664,
          "image_size": 224,
          "initializer_factor": 1.0,
          "initializer_range": 0.02,
          "intermediate_size": 8192,
          "layer_norm_eps": 1e-05,
          "model_type": "clip_vision_model",
          "num_attention_heads": 16,
          "num_channels": 3,
          "num_hidden_layers": 48,
          "patch_size": 14,
          "projection_dim": 512,
          "transformers_version": "4.27.2"
        }
        vision_config = CLIPVisionConfig(**config)

        self.projection_dim = 1280
        self.vision_embed_dim = 1664

        self.vision_model = CLIPVisionTransformer(vision_config)
        self.visual_projection = nn.Linear(self.vision_embed_dim, self.projection_dim, bias=False)

    def forward(self, x):
        return self.get_image_features(pixel_values=x)

    def get_image_features(
            self,
            pixel_values: Optional[torch.FloatTensor] = None,
            output_attentions: Optional[bool] = False,
            output_hidden_states: Optional[bool] = False,
            return_dict: Optional[bool] = True,
        ) -> torch.FloatTensor:

        vision_outputs = self.vision_model(
            pixel_values=pixel_values,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        pooled_output = vision_outputs[1]  # pooled_output
        image_features = self.visual_projection(pooled_output)

        return image_features



In [25]:
model_to_hub = VitImageFeatureExtractor()

In [26]:
model_to_hub.visual_projection = model.feature_extractor.visual_projection

In [27]:
model_to_hub.vision_model = model.feature_extractor.vision_model

In [2]:
model_to_hub.push_to_hub("ViT-bigG-14-laion2B-39B-b160k")

NameError: name 'model_to_hub' is not defined

In [2]:
model = VitImageFeatureExtractor.from_pretrained("dbuos/ViT-bigG-14-laion2B-39B-b160k")

config.json not found in HuggingFace Hub.


In [3]:
class Extractor(torch.nn.Module):
    def __init__(self, vision_model):
        super().__init__()
        self.vision_model = vision_model

    def forward(self, x):
        img_a, img_b = x
        return self.vision_model.get_image_features(pixel_values=img_a), self.vision_model.get_image_features(
            pixel_values=img_b)

In [4]:
model2 = Extractor(model)

In [5]:
#Only model weights in GPU: 7.9 GB
#Batch of 4 images pairs forward pass: +13.5GB
#Batch of 4 images pairs backward pass: +5GB
#Fordward pass time GPU batch 4 pairs: 300 ms
#forward + Backward  pass time CPU batch 4 pairs: 13 segs + 40 segs

In [6]:
import torch
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
model2.to(device)
model2.train()
images_batch_a = torch.rand(4, 3, 224, 224).to(device)
images_batch_b = torch.rand(4, 3, 224, 224).to(device)

In [7]:
out_layer = nn.Linear(1280*2, 1)
out_layer.to(device)

Linear(in_features=2560, out_features=1, bias=True)

In [8]:
from torch.nn import BCEWithLogitsLoss
loss = BCEWithLogitsLoss()
y_true = torch.tensor([0.0, 1.1, 0.0, 1.0]).to(device)

torch.float32

In [9]:
import time
start = time.time()
out = model2((images_batch_a, images_batch_b))
out = torch.cat(out, dim=1)
out = out_layer(out)
end = time.time()
milliseconds = (end - start) * 1000
print(f"Time: {milliseconds} ms")

Time: 13930.768013000488 ms


In [10]:
out = out.squeeze()


In [11]:
loss_val = loss(out, y_true)

In [12]:
loss_val.backward()

In [17]:
#Clean cuda cache
del out
torch.cuda.empty_cache()

NameError: name 'out' is not defined

In [12]:

config = {"attention_dropout": 0.0,
  "dropout": 0.0,
  "hidden_act": "gelu",
  "hidden_size": 1664,
  "image_size": 224,
  "initializer_factor": 1.0,
  "initializer_range": 0.02,
  "intermediate_size": 8192,
  "layer_norm_eps": 1e-05,
  "model_type": "clip_vision_model",
  "num_attention_heads": 16,
  "num_channels": 3,
  "num_hidden_layers": 48,
  "patch_size": 14,
  "projection_dim": 512,
  "transformers_version": "4.27.2"}
CLIPVisionConfig(**config)

CLIPVisionConfig {
  "attention_dropout": 0.0,
  "dropout": 0.0,
  "hidden_act": "gelu",
  "hidden_size": 1664,
  "image_size": 224,
  "initializer_factor": 1.0,
  "initializer_range": 0.02,
  "intermediate_size": 8192,
  "layer_norm_eps": 1e-05,
  "model_type": "clip_vision_model",
  "num_attention_heads": 16,
  "num_channels": 3,
  "num_hidden_layers": 48,
  "patch_size": 14,
  "projection_dim": 512,
  "transformers_version": "4.27.2"
}

In [16]:
type(model.vision_model.encoder)

transformers.models.clip.modeling_clip.CLIPEncoder

In [ ]:
from transformers.models.clip.modeling_clip import CLIPEncoder